# Setup

In [ ]:
!pip install transformers datasets torch
!pip install sentencepiece
!pip install accelerate -U
!pip install textattack
!pip3 install textattack[tensorflow]
!pip install --upgrade tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.8/436.8 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 387.2/387.2 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 769.7/769.7 kB 15.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.3/143.3 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 25.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━ 376.4/511.7 MB 146.1 MB/s eta 0:00:01
ERROR: Exception:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/urllib3/response.py", line 438, in _error_catcher
    yield
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/urllib3/response.py", line 561, in read
    data = self._fp_read(amt) if not fp_closed else b""
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/urllib3/response.py", line 527, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/cachecontrol/filewrapper.py", line 90, in read
    data = self.__fp.read(amt)
  File "/usr/lib/python3.10/http/client.py", line 466, in read
    s = self.fp.read(amt)
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
  File "/usr/lib/python3.10/ssl.py", line 1303, in recv_into
    return self.read(nbytes,

In [ ]:
import torch
from datasets import load_dataset
from transformers import DebertaV2ForSequenceClassification, DebertaV2Tokenizer, Trainer, TrainingArguments
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
if torch.cuda.is_available():
    print(f"GPU available: {torch.cuda.get_device_name(0)}")
else:
    print("GPU not available")

GPU available: Tesla T4



# Before fine tuning
Additional classification head

Accuracy = 32%

In [ ]:
# Load the tokenizer and model
tokenizer = DebertaV2Tokenizer.from_pretrained('microsoft/deberta-v3-xsmall')
model = DebertaV2ForSequenceClassification.from_pretrained('microsoft/deberta-v3-xsmall', num_labels = 3)

# Load the RTE dataset
dataset = load_dataset("glue", "mnli")

# Prepare the validation set
validation_set = dataset['validation_matched']

# Function to evaluate the model on the validation set
def evaluate_model(model, tokenizer, validation_set):
    model.eval()  # Set the model to evaluation mode

    correct_predictions = 0
    total_predictions = 0

    with torch.no_grad():
        for item in validation_set:
            inputs = tokenizer(item['premise'], item['hypothesis'], return_tensors="pt", truncation=True, padding=True)
            outputs = model(**inputs)
            predictions = torch.argmax(outputs.logits, dim=1)
            correct_predictions += (predictions == item['label']).sum().item()
            total_predictions += 1

    accuracy = correct_predictions / total_predictions
    return accuracy

# Calculate accuracy
accuracy = evaluate_model(model, tokenizer, validation_set)
print(f"Accuracy: {accuracy}")


Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-xsmall and are newly initialized: ['pooler.dense.bias', 'classifier.weight', 'pooler.dense.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Accuracy: 0.31859398879266426


In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

# Fine tuning

In [ ]:
model_name = "microsoft/deberta-v3-xsmall"
tokenizer = DebertaV2Tokenizer.from_pretrained(model_name)
model = DebertaV2ForSequenceClassification.from_pretrained(model_name, num_labels=3)

dataset = load_dataset("glue", "mnli")

def tokenize_function(examples):
    return tokenizer(examples["premise"], examples["hypothesis"], truncation=True, padding="max_length", max_length=128)
tokenized_datasets = dataset.map(tokenize_function, batched=True)

training_args = TrainingArguments(
    output_dir="./results/mnli",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    num_train_epochs=1,
    weight_decay=0.01,
)

# training_args = TrainingArguments(
#     output_dir="./results/rte",
#     evaluation_strategy="steps",
#     eval_steps=50,
#     learning_rate=1e-5,
#     per_device_train_batch_size=16,
#     num_train_epochs=3,
#     weight_decay=0.02,
#     load_best_model_at_end=True,
#     metric_for_best_model="eval_loss",
#     greater_is_better=False,
# )
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation_matched"],
    tokenizer=tokenizer
)

trainer.train()

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/241M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-xsmall and are newly initialized: ['classifier.bias', 'pooler.dense.bias', 'pooler.dense.weight', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Map:   0%|          | 0/9847 [00:00<?, ? examples/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss
1,0.396800,0.378224


TrainOutput(global_step=49088, training_loss=0.44749604322919634, metrics={'train_runtime': 6701.5238, 'train_samples_per_second': 58.599, 'train_steps_per_second': 7.325, 'total_flos': 6467435954302464.0, 'train_loss': 0.44749604322919634, 'epoch': 1.0})

# Saving model and tokeniser to google drive

In [ ]:
save_directory = "/content/drive/My Drive/mnli-deberta-xsmall"

In [ ]:
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

('/content/drive/My Drive/mnli-deberta-xsmall/tokenizer_config.json',
 '/content/drive/My Drive/mnli-deberta-xsmall/special_tokens_map.json',
 '/content/drive/My Drive/mnli-deberta-xsmall/spm.model',
 '/content/drive/My Drive/mnli-deberta-xsmall/added_tokens.json')

# Loading model/tokeniser from google drive

In [ ]:
from transformers import DebertaV2Tokenizer, DebertaV2ForSequenceClassification

model = DebertaV2ForSequenceClassification.from_pretrained(save_directory)
tokenizer = DebertaV2Tokenizer.from_pretrained(save_directory)

In [ ]:
dataset = load_dataset("glue", "mnli")
validation_dataset = dataset["validation_matched"]

def tokenize_function(examples):
    return tokenizer(examples["premise"], examples["hypothesis"], truncation=True, padding="max_length", max_length=128)

tokenized_validation_dataset = validation_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/9815 [00:00<?, ? examples/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

# Calculating Accuracy of fine tuned model

Accuracy = 87%

In [ ]:
from transformers import Trainer
import numpy as np

trainer = Trainer(model=model)

raw_pred, _, _ = trainer.predict(tokenized_validation_dataset)

# Apply softmax and then argmax to get the most likely class
predictions = np.argmax(raw_pred, axis=1)

In [ ]:
import numpy as np

# Apply softmax and then argmax to get the most likely class
predictions = np.argmax(raw_pred, axis=1)

In [ ]:
true_labels = validation_dataset["label"]

# Analyze the results
for i, (pred, label) in enumerate(zip(predictions, true_labels)):
    if pred != label:
        print(f"Index: {i}, Predicted: {pred}, Actual: {label}")
        print(f"Sentence 1: {validation_dataset[i]['premise']}")
        print(f"Sentence 2: {validation_dataset[i]['hypothesis']}")
        print("-----")

In [ ]:
accuracy = np.mean(predictions == true_labels)
print(f"Validation Accuracy: {accuracy}")

Validation Accuracy: 0.87396841569027
Validation Accuracy: 0.87396841569027


# Attack 1 Textfooler

In [ ]:
from textattack.models.wrappers import HuggingFaceModelWrapper
from textattack.datasets import HuggingFaceDataset
from textattack.attack_recipes import TextFoolerJin2019
from textattack import Attacker, AttackArgs
from textattack.loggers import CSVLogger

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
model_wrapper = HuggingFaceModelWrapper(model, tokenizer)
dataset = HuggingFaceDataset("glue", "mnli", split="validation_matched")
attack = TextFoolerJin2019.build(model_wrapper)
attack_args = AttackArgs(
    num_examples=200,
    log_to_csv="log.csv",
    checkpoint_interval=5,
    checkpoint_dir="checkpoints",
    disable_stdout=False
)

textattack: Loading datasets dataset glue, subset mnli, split validation_matched.
textattack: Unknown if model of class <class 'transformers.models.deberta_v2.modeling_deberta_v2.DebertaV2ForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [ ]:
attacker = Attacker(attack, dataset, attack_args)
attacker.attack_dataset()

textattack: Logging to CSV at path log.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:   0%|          | 1/200 [01:02<3:26:37, 62.30s/it]

--------------------------------------------- Result 1 ---------------------------------------------
[[Neutral (98%)]] --> [[Contradiction (89%)]]

[[[[Premise]]]]: The new rights are nice enough
[[[[Hypothesis]]]]: Everyone really [[likes]] the newest benefits 

[[[[Premise]]]]: The new rights are nice enough
[[[[Hypothesis]]]]: Everyone really [[amateurs]] the newest benefits 




[Succeeded / Failed / Skipped / Total] 2 / 0 / 0 / 2:   1%|          | 2/200 [01:03<1:44:48, 31.76s/it]

...

--------------------------------------------- Result 199 ---------------------------------------------
[[Entailment (95%)]] --> [[Neutral (84%)]]

[[[[Premise]]]]: Such multicolored reef dwellers as the parrotfish and French angelfish, along with weirdly shaped coral, crawfish, or turtles hiding in crevices, can be yours for the viewing in these clear waters where visibility of 30 m (100 ft) is common.
[[[[Hypothesis]]]]: Because [[visibility]] is so good, you can see French angelfish, crawfish, and other reef dwellers.

[[[[Premise]]]]: Such multicolored reef dwellers as the parrotfish and French angelfish, along with weirdly shaped coral, crawfish, or turtles hiding in crevices, can be yours for the viewing in these clear waters where visibility of 30 m (100 ft) is common.
[[[[Hypothesis]]]]: Because [[notoriety]] is so good, you can see French angelfish, crawfish, and other reef dwellers.




[Succeeded / Failed / Skipped / Total] 158 / 18 / 24 / 200: 100%|██████████| 200/200 [04:31<00:00,  1.36s/it]textattack: Saving checkpoint under "checkpoints/1701054170452.ta.chkpt" at 2023-11-27 03:02:50 after 200 attacks.


--------------------------------------------- Result 200 ---------------------------------------------
[[Entailment (99%)]] --> [[Contradiction (96%)]]

[[[[Premise]]]]: Benchmarked by U.S.
[[[[Hypothesis]]]]: Benchmark in [[America]].

[[[[Premise]]]]: Benchmarked by U.S.
[[[[Hypothesis]]]]: Benchmark in [[Latina]].







[Succeeded / Failed / Skipped / Total] 158 / 18 / 24 / 200: 100%|██████████| 200/200 [04:31<00:00,  1.36s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 158    |
| Number of failed attacks:     | 18     |
| Number of skipped attacks:    | 24     |
| Original accuracy:            | 88.0%  |
| Accuracy under attack:        | 9.0%   |
| Attack success rate:          | 89.77% |
| Average perturbed word %:     | 6.58%  |
| Average num. words per input: | 29.96  |
| Avg num queries:              | 54.78  |
+-------------------------------+--------+


# Attack 2 - Deepwordbug

In [ ]:
from textattack.attack_recipes import DeepWordBugGao2018

In [ ]:
model_wrapper = HuggingFaceModelWrapper(model, tokenizer)
dataset = HuggingFaceDataset("glue", "mnli", split="validation_matched")
attack = DeepWordBugGao2018.build(model_wrapper)
attack_args = AttackArgs(
    num_examples=200,
    log_to_csv="log.csv",
    checkpoint_interval=5,
    checkpoint_dir="checkpoints",
    disable_stdout=False
)

textattack: Loading datasets dataset glue, subset mnli, split validation_matched.
textattack: Unknown if model of class <class 'transformers.models.deberta_v2.modeling_deberta_v2.DebertaV2ForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [ ]:
attacker = Attacker(attack, dataset, attack_args)
attacker.attack_dataset()

textattack: Logging to CSV at path log.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  CompositeTransformation(
    (0): WordSwapNeighboringCharacterSwap(
        (random_one):  True
      )
    (1): WordSwapRandomCharacterSubstitution(
        (random_one):  True
      )
    (2): WordSwapRandomCharacterDeletion(
        (random_one):  True
      )
    (3): WordSwapRandomCharacterInsertion(
        (random_one):  True
      )
    )
  (constraints): 
    (0): LevenshteinEditDistance(
        (max_edit_distance):  30
        (compare_against_original):  True
      )
    (1): RepeatModification
    (2): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:   0%|          | 1/200 [00:00<01:20,  2.46it/s]

--------------------------------------------- Result 1 ---------------------------------------------
[[Neutral (98%)]] --> [[Contradiction (97%)]]

[[[[Premise]]]]: The new rights are nice enough
[[[[Hypothesis]]]]: Everyone really [[likes]] the newest benefits 

[[[[Premise]]]]: The new rights are nice enough
[[[[Hypothesis]]]]: Everyone really [[lices]] the newest benefits 




[Succeeded / Failed / Skipped / Total] 2 / 0 / 0 / 2:   1%|          | 2/200 [00:01<02:18,  1.43it/s]

...

--------------------------------------------- Result 58 ---------------------------------------------
[[Contradiction (100%)]] --> [[Neutral (64%)]]

[[[[Premise]]]]: Oh, what a [[fool]] I feel! 
[[[[Hypothesis]]]]: I am beyond proud.

[[[[Premise]]]]: Oh, what a [[fofol]] I feel! 
[[[[Hypothesis]]]]: I am beyond proud.




[Succeeded / Failed / Skipped / Total] 52 / 2 / 5 / 59:  30%|██▉       | 59/200 [00:33<01:20,  1.74it/s]

--------------------------------------------- Result 59 ---------------------------------------------
[[Contradiction (99%)]] --> [[Entailment (69%)]]

[[[[Premise]]]]: She has [[exchanged]] a hollow life for a heightened life, and has tried to comprehend all its turns, get its possibilities.
[[[[Hypothesis]]]]: She has chose to live a hollow life.

[[[[Premise]]]]: She has [[exchangVed]] a hollow life for a heightened life, and has tried to comprehend all its turns, get its possibilities.
[[[[Hypothesis]]]]: She has chose to live a hollow life.




[Succeeded / Failed / Skipped / Total] 53 / 2 / 5 / 60:  30%|███       | 60/200 [00:35<01:23,  1.68it/s]textattack: Saving checkpoint under "checkpoints/1701054209562.ta.chkpt" at 2023-11-27 03:03:29 after 60 attacks.


--------------------------------------------- Result 60 ---------------------------------------------
[[Neutral (97%)]] --> [[Contradiction (71%)]]

[[[[Premise]]]]: well the first thing for me is i wonder i see a couple of different ways of talking about what privacy is um if privacy is something that disturbs your private state i mean an invasion of privacy is something that disturbs your private state that's one thing and if privacy is something that comes into your private state and extracts information from it in other words finds something out about you that's another and the first kind of invasion of the first type of privacy [[seems]] invaded to me in very much everyday in this country but in the [[second]] type at least overtly uh where someone comes in and uh finds out information about you that should be private uh does not seem uh um obviously everyday
[[[[Hypothesis]]]]: All invasions of privacy should be severely [[punished]], because it will [[teach]] the [[criminals]] t

# Attack 3 - TextBugger

In [ ]:
import torch
from datasets import load_dataset
from transformers import DebertaV2ForSequenceClassification, DebertaV2Tokenizer, Trainer, TrainingArguments
import numpy as np

save_directory = "/content/drive/My Drive/finetuned_models/mnli-deberta-xsmall"
model = DebertaV2ForSequenceClassification.from_pretrained(save_directory)
tokenizer = DebertaV2Tokenizer.from_pretrained(save_directory)

from textattack.models.wrappers import HuggingFaceModelWrapper
from textattack.datasets import HuggingFaceDataset
from textattack.attack_recipes import TextBuggerLi2018
from textattack import Attacker, AttackArgs
from textattack.loggers import CSVLogger

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
model_wrapper = HuggingFaceModelWrapper(model, tokenizer)
dataset = HuggingFaceDataset("glue", "mnli", split="validation_matched")
attack = TextBuggerLi2018.build(model_wrapper)
attack_args = AttackArgs(
    num_examples=200,
    log_to_csv="log_textbugger.csv",
    checkpoint_interval=5,
    checkpoint_dir="checkpoints",
    disable_stdout=False
)

Generating train split:   0%|          | 0/392702 [00:00<?, ? examples/s]

Generating validation_matched split:   0%|          | 0/9815 [00:00<?, ? examples/s]

Generating validation_mismatched split:   0%|          | 0/9832 [00:00<?, ? examples/s]

Generating test_matched split:   0%|          | 0/9796 [00:00<?, ? examples/s]

Generating test_mismatched split:   0%|          | 0/9847 [00:00<?, ? examples/s]

textattack: Loading datasets dataset glue, subset mnli, split validation_matched.
textattack: Unknown if model of class <class 'transformers.models.deberta_v2.modeling_deberta_v2.DebertaV2ForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [ ]:
attacker = Attacker(attack, dataset, attack_args)
attacker.attack_dataset()

textattack: Logging to CSV at path log_textbugger.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  CompositeTransformation(
    (0): WordSwapRandomCharacterInsertion(
        (random_one):  True
      )
    (1): WordSwapRandomCharacterDeletion(
        (random_one):  True
      )
    (2): WordSwapNeighboringCharacterSwap(
        (random_one):  True
      )
    (3): WordSwapHomoglyphSwap
    (4): WordSwapEmbedding(
        (max_candidates):  5
        (embedding):  WordEmbedding
      )
    )
  (constraints): 
    (0): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8
        (window_size):  inf
        (skip_text_shorter_than_window):  False
        (compare_against_original):  True
      )
    (1): RepeatModification
    (2): StopwordModification
  (is_black_box):  True
) 





  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 1/200 [00:10<34:54, 10.52s/it]

[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:   0%|          | 1/200 [00:10<35:03, 10.57s/it]

--------------------------------------------- Result 1 ---------------------------------------------
[[Neutral (98%)]] --> [[Contradiction (88%)]]

[[[[Premise]]]]: The new rights are nice enough
[[[[Hypothesis]]]]: Everyone really [[likes]] the newest benefits 

[[[[Premise]]]]: The new rights are nice enough
[[[[Hypothesis]]]]: Everyone really [[iike]] the newest benefits 






[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:   1%|          | 2/200 [00:12<21:22,  6.48s/it]

[Succeeded / Failed / Skipped / Total] 2 / 0 / 0 / 2:   1%|          | 2/200 [00:12<21:26,  6.50s/it]

...

--------------------------------------------- Result 199 ---------------------------------------------
[[Entailment (95%)]] --> [[Neutral (79%)]]

[[[[Premise]]]]: Such multicolored reef dwellers as the parrotfish and [[French]] angelfish, along with weirdly shaped coral, [[crawfish]], or turtles hiding in crevices, can be yours for the viewing in these clear waters where visibility of 30 m (100 ft) is common.
[[[[Hypothesis]]]]: Because visibility is so good, you can see French angelfish, crawfish, and other reef dwellers.

[[[[Premise]]]]: Such multicolored reef dwellers as the parrotfish and [[Fⲅench]] angelfish, along with weirdly shaped coral, [[crayfish]], or turtles hiding in crevices, can be yours for the viewing in these clear waters where visibility of 30 m (100 ft) is common.
[[[[Hypothesis]]]]: Because visibility is so good, you can see French angelfish, crawfish, and other reef dwellers.






[Succeeded / Failed / Skipped / Total] 162 / 13 / 24 / 199: 100%|██████████| 200/200 [03:10<00:00,  1.05it/s]

[Succeeded / Failed / Skipped / Total] 163 / 13 / 24 / 200: 100%|██████████| 200/200 [03:10<00:00,  1.05it/s]textattack: Saving checkpoint under "checkpoints/1701100027979.ta.chkpt" at 2023-11-27 15:47:07 after 200 attacks.


--------------------------------------------- Result 200 ---------------------------------------------
[[Entailment (99%)]] --> [[Contradiction (38%)]]

[[[[Premise]]]]: Benchmarked by U.S.
[[[[Hypothesis]]]]: [[Benchmark]] in America.

[[[[Premise]]]]: Benchmarked by U.S.
[[[[Hypothesis]]]]: [[Benϲhmark]] in America.







[Succeeded / Failed / Skipped / Total] 163 / 13 / 24 / 200: 100%|██████████| 200/200 [03:10<00:00,  1.05it/s]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 163    |
| Number of failed attacks:     | 13     |
| Number of skipped attacks:    | 24     |
| Original accuracy:            | 88.0%  |
| Accuracy under attack:        | 6.5%   |
| Attack success rate:          | 92.61% |
| Average perturbed word %:     | 10.66% |
| Average num. words per input: | 29.96  |
| Avg num queries:              | 44.36  |
+-------------------------------+--------+


# Attack 4 - PWWS

In [ ]:
import torch
from datasets import load_dataset
from transformers import DebertaV2ForSequenceClassification, DebertaV2Tokenizer, Trainer, TrainingArguments
import numpy as np

save_directory = "/content/drive/My Drive/finetuned_models/mnli-deberta-xsmall"
model = DebertaV2ForSequenceClassification.from_pretrained(save_directory)
tokenizer = DebertaV2Tokenizer.from_pretrained(save_directory)

from textattack.models.wrappers import HuggingFaceModelWrapper
from textattack.datasets import HuggingFaceDataset
from textattack.attack_recipes import PWWSRen2019
from textattack import Attacker, AttackArgs
from textattack.loggers import CSVLogger

In [ ]:
model_wrapper = HuggingFaceModelWrapper(model, tokenizer)
dataset = HuggingFaceDataset("glue", "mnli", split="validation_matched")
attack = PWWSRen2019.build(model_wrapper)
attack_args = AttackArgs(
    num_examples=200,
    log_to_csv="log_PWWS.csv",
    checkpoint_interval=5,
    checkpoint_dir="checkpoints",
    disable_stdout=False
)

textattack: Loading datasets dataset glue, subset mnli, split validation_matched.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
textattack: CSVLogger exiting without calling flush().
[Succeeded / Failed / Skipped / Total] 0 / 0 / 1 / 1:   1%|          | 2/200 [13:35<22:25:37, 407.76s/it]
textattack: CSVLogger exiting without calling flush().
[Succeeded / Failed / Skipped / Total] 0 / 0 / 1 / 1:   1%|          | 2/200 [06:40<11:00:26, 200.13s/it]
textattack: Unknown if model of class <class 'transformers.models.deberta_v2.modeling_deberta_v2.DebertaV2ForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [ ]:
attacker = Attacker(attack, dataset, attack_args)
attacker.attack_dataset()

textattack: Logging to CSV at path log_PWWS.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  weighted-saliency
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapWordNet
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:   0%|          | 1/200 [00:01<04:39,  1.40s/it]

--------------------------------------------- Result 1 ---------------------------------------------
[[Neutral (98%)]] --> [[Contradiction (95%)]]

[[[[Premise]]]]: The new rights are [[nice]] enough
[[[[Hypothesis]]]]: Everyone really likes the newest benefits 

[[[[Premise]]]]: The new rights are [[squeamish]] enough
[[[[Hypothesis]]]]: Everyone really likes the newest benefits 




[Succeeded / Failed / Skipped / Total] 2 / 0 / 0 / 2:   1%|          | 2/200 [00:03<05:46,  1.75s/it]

...

--------------------------------------------- Result 199 ---------------------------------------------
[[Entailment (95%)]] --> [[Neutral (53%)]]

[[[[Premise]]]]: Such multicolored reef dwellers as the parrotfish and French angelfish, along with weirdly shaped coral, [[crawfish]], or turtles hiding in crevices, can be yours for the viewing in these clear waters where visibility of 30 m (100 ft) is common.
[[[[Hypothesis]]]]: Because visibility is so good, you can see French angelfish, crawfish, and other reef dwellers.

[[[[Premise]]]]: Such multicolored reef dwellers as the parrotfish and French angelfish, along with weirdly shaped coral, [[retreat]], or turtles hiding in crevices, can be yours for the viewing in these clear waters where visibility of 30 m (100 ft) is common.
[[[[Hypothesis]]]]: Because visibility is so good, you can see French angelfish, crawfish, and other reef dwellers.




[Succeeded / Failed / Skipped / Total] 168 / 8 / 24 / 200: 100%|██████████| 200/200 [09:57<00:00,  2.99s/it]textattack: Saving checkpoint under "checkpoints/1701100667908.ta.chkpt" at 2023-11-27 15:57:47 after 200 attacks.


--------------------------------------------- Result 200 ---------------------------------------------
[[Entailment (99%)]] --> [[Contradiction (91%)]]

[[[[Premise]]]]: Benchmarked by [[U]].[[S]].
[[[[Hypothesis]]]]: Benchmark in America.

[[[[Premise]]]]: Benchmarked by [[uranium]].[[mho]].
[[[[Hypothesis]]]]: Benchmark in America.







[Succeeded / Failed / Skipped / Total] 168 / 8 / 24 / 200: 100%|██████████| 200/200 [09:57<00:00,  2.99s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 168    |
| Number of failed attacks:     | 8      |
| Number of skipped attacks:    | 24     |
| Original accuracy:            | 88.0%  |
| Accuracy under attack:        | 4.0%   |
| Attack success rate:          | 95.45% |
| Average perturbed word %:     | 6.57%  |
| Average num. words per input: | 29.96  |
| Avg num queries:              | 187.03 |
+-------------------------------+--------+


# Attack 5 - Improved Genetic Algorithm

In [ ]:
import torch
from datasets import load_dataset
from transformers import DebertaV2ForSequenceClassification, DebertaV2Tokenizer, Trainer, TrainingArguments
import numpy as np

save_directory = "/content/drive/My Drive/finetuned_models/mnli-deberta-xsmall"
model = DebertaV2ForSequenceClassification.from_pretrained(save_directory)
tokenizer = DebertaV2Tokenizer.from_pretrained(save_directory)

from textattack.models.wrappers import HuggingFaceModelWrapper
from textattack.datasets import HuggingFaceDataset
from textattack.attack_recipes import PSOZang2020
from textattack import Attacker, AttackArgs
from textattack.loggers import CSVLogger

In [ ]:
model_wrapper = HuggingFaceModelWrapper(model, tokenizer)
dataset = HuggingFaceDataset("glue", "mnli", split="validation_matched")
attack = PSOZang2020.build(model_wrapper)
attack_args = AttackArgs(
    num_examples=200,
    log_to_csv="mnli_log_PSO.csv",
    checkpoint_interval=5,
    checkpoint_dir="checkpoints",
    disable_stdout=False
)

textattack: Loading datasets dataset glue, subset mnli, split validation_matched.
textattack: Downloading https://textattack.s3.amazonaws.com/transformations/hownet/word_candidates_sense.pkl.



  0%|          | 0.00/8.39M [00:00<?, ?B/s]


  0%|          | 34.8k/8.39M [00:00<00:25, 322kB/s]


  3%|▎         | 226k/8.39M [00:00<00:07, 1.10MB/s]


 11%|█         | 905k/8.39M [00:00<00:02, 3.20MB/s]


 33%|███▎      | 2.78M/8.39M [00:00<00:00, 8.70MB/s]


100%|██████████| 8.39M/8.39M [00:00<00:00, 12.9MB/s]
textattack: Copying /root/.cache/textattack/tmpcev5069w.zip to /root/.cache/textattack/transformations/hownet/word_candidates_sense.pkl.
textattack: Successfully saved transformations/hownet/word_candidates_sense.pkl to cache.
textattack: CSVLogger exiting without calling flush().
[Succeeded / Failed / Skipped / Total] 4 / 0 / 0 / 4:   2%|▏         | 4/200 [08:33<6:59:28, 128.41s/it]
[Succeeded / Failed / Skipped / Total] 4 / 0 / 0 / 4:   2%|▏         | 4/200 [03:04<2:30:24, 46.04s/it

In [ ]:
attacker = Attacker(attack, dataset, attack_args)
attacker.attack_dataset()

textattack: Logging to CSV at path mnli_log_IGA.csv


Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  60
    (max_iters):  20
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapHowNet(
    (max_candidates):  -1
  )
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
    (2): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (columns_to_ignore):  {'premise'}
      )
  (is_black_box):  True
) 



  0%|          | 0/200 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/72.9M [00:00<?, ?B/s]

2023-11-29 19:53:33,377 SequenceTagger predicts: Dictionary with 19 tags: <unk>, NOUN, VERB, PUNCT, ADP, DET, PROPN, PRON, ADJ, ADV, CCONJ, PART, NUM, AUX, INTJ, SYM, X, <START>, <STOP>


...

--------------------------------------------- Result 199 ---------------------------------------------
[[Entailment (95%)]] --> [[Neutral (91%)]]

[[[[Premise]]]]: Such multicolored reef dwellers as the parrotfish and French angelfish, along with weirdly shaped coral, crawfish, or turtles hiding in crevices, can be yours for the viewing in these clear waters where visibility of 30 m (100 ft) is common.
[[[[Hypothesis]]]]: Because visibility is so [[good]], you can see French angelfish, crawfish, and other reef dwellers.

[[[[Premise]]]]: Such multicolored reef dwellers as the parrotfish and French angelfish, along with weirdly shaped coral, crawfish, or turtles hiding in crevices, can be yours for the viewing in these clear waters where visibility of 30 m (100 ft) is common.
[[[[Hypothesis]]]]: Because visibility is so [[humane]], you can see French angelfish, crawfish, and other reef dwellers.




[Succeeded / Failed / Skipped / Total] 96 / 80 / 24 / 200: 100%|██████████| 200/200 [07:05<00:00,  2.13s/it]textattack: Saving checkpoint under "checkpoints/1701288035724.ta.chkpt" at 2023-11-29 20:00:35 after 200 attacks.


--------------------------------------------- Result 200 ---------------------------------------------
[[Entailment (99%)]] --> [[[FAILED]]]

[[[[Premise]]]]: Benchmarked by U.S.
[[[[Hypothesis]]]]: Benchmark in America.







[Succeeded / Failed / Skipped / Total] 96 / 80 / 24 / 200: 100%|██████████| 200/200 [07:05<00:00,  2.13s/it]


+-------------------------------+---------+
| Attack Results                |         |
+-------------------------------+---------+
| Number of successful attacks: | 96      |
| Number of failed attacks:     | 80      |
| Number of skipped attacks:    | 24      |
| Original accuracy:            | 88.0%   |
| Accuracy under attack:        | 40.0%   |
| Attack success rate:          | 54.55%  |
| Average perturbed word %:     | 5.61%   |
| Average num. words per input: | 29.96   |
| Avg num queries:              | 1013.13 |
+-------------------------------+---------+
